### Goal for This notebook:
1. Feature Engineernig 
2. Clean Data
3. Feature Extraction
4. Visualise Data


### Load data and liberary
* Here we load basic liberary 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import plotly.express as px
### so that u dont have warnings
from warnings import filterwarnings
import plotly.graph_objs as go
from plotly.offline import iplot
filterwarnings('ignore')
# Custom colors
class color:
    S = '\033[1m' + '\033[93m'
    E = '\033[0m'
    
my_colors = ["#E7C84B", "#4EE4EA", "#4EA9EA", "#242179", "#AB51E9", "#E051E9"]
print(color.S+"Notebook Color Scheme:"+color.E)
sns.palplot(sns.color_palette(my_colors))

# Set Style
sns.set_style("white")
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.spines.left'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
plt.rcParams.update({'font.size': 17})

In [ ]:
#### Read in the training data

In [ ]:
# Read in the training data
train_df = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")

# Print some useful information
print(color.S+"Train Data has:"+color.E, "{:,}".format(train_df.shape[0]), "observations.", "\n" +
      color.S+"Number of Missing Values:"+color.E, train_df.isna().sum()[0], "\n" +
      "\n" +
      color.S+"Head of Training Data:"+color.E)
train_df.head(5)

### Read in the Test data

In [ ]:
test_df = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")

# Print some useful information
print(color.S+"Test Data has:"+color.E, "{:,}".format(test_df.shape[0]), "observations.", "\n" +
      color.S+"Number of Missing Values:"+color.E, test_df.isna().sum()[0], "\n" +
      "\n" +
      color.S+"Head of Training Data:"+color.E)
test_df.head(5).style.background_gradient(cmap = 'Reds')

### Work with Train Data

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.dtypes

In [ ]:
feature_na=[feature for feature in train_df.columns if train_df[feature].isnull().sum()>0]
feature_na

 #### getting all NAN features

In [ ]:
#% of missing values
for feature in feature_na:
    print('{} has {} % missing values'.format(feature,np.round(trian_df[feature].isnull().sum()/len(train_df)*100,4)))

In [ ]:
train_df['loss'].unique()

### Showing Basics Statistics
Now that you’ve seen what data types are in your dataset, it’s time to get an overview of the values each column contains. You can do this with .describe():


In [ ]:
train_df.describe().style.background_gradient(cmap = 'Reds')

In [ ]:
train_df.max()

In [ ]:
train_df.describe().T.style.bar(subset=['mean'], color='red')\
                            .background_gradient(subset=['std'], cmap='Reds')\
                            .background_gradient(subset=['50%'], cmap='coolwarm')

*  This data haven't any decimal value all value float

In [ ]:
# Skewness of the distribution

print(train_df.skew())


### Data Interaction
•Correlation
*  Correlation tells relation between two attributes.
*  Correlation requires continous data. Hence, ignore categorical data
* Calculates pearson co-efficient for all combinations


In [ ]:
data_corr = train_df.corr()
data_corr.style.background_gradient(cmap = 'copper')

In [ ]:
sns.scatterplot(x='f1', y='f2', hue='loss', data=train_df)

In [ ]:
sns.scatterplot(x=train_df['f1'], y=train_df['loss'])

In [ ]:
sns.regplot(x=train_df['f1'], y=train_df['loss'])

### calculate avg f1 and loss

In [ ]:
train_df.groupby('f1')['loss'].mean().nlargest(20).plot.bar()

##### Loss distribution

In [ ]:
sns.set_style(style='whitegrid')
sns.distplot(train_df['loss'])

In [ ]:
plt.figure(figsize=(10,7))
chains=train_df['loss'].value_counts()[0:20]
sns.barplot(x=chains,y=chains.index,palette='deep')
plt.title("Most loss ")
plt.xlabel("Number of outlets") 

In [ ]:
x=train_df['f1'].value_counts()
labels=['accepted','not accepted']

In [ ]:
fig = px.scatter_3d(train_df.iloc[:500], x='f0', y='f1', z='f2',
                    color='loss')
fig.show()

In [ ]:
plt.figure(figsize=(20,12))
train_df['loss'].value_counts().nlargest(20).plot.bar(color='red')
plt.gcf().autofmt_xdate()

#### How many types ofLosses we have?

In [ ]:
train_df['loss'].isna().sum()

In [ ]:
len(train_df['loss'].unique())

In [ ]:
trace1 = go.Bar( 
        x = train_df.groupby('f1')['f2'].max().nlargest(12).index,
        y = train_df.groupby('f3')['f4'].max().nlargest(12),
        name= 'loss')
iplot([trace1])

In [ ]:
fig = px.box(train_df.head(500),x='f1',y='loss')
fig.show()

In [ ]:
sns.kdeplot(train_df["loss"]);

In [ ]:
train_df.head()

In [ ]:
X = train_df.iloc[:, [2, 3]].values
y = train_df.iloc[:, 4].values

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean')
imputer=imputer.fit(X[:,1:3])
X[:,1:3]=imputer.transform(X[:,1:3])

In [ ]:
#Encoding categorical data
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder_X=LabelEncoder()
X[:,0]=labelencoder_X.fit_transform(X[:,0])

In [ ]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(categories='auto'), [0])],remainder='passthrough')
X = ct.fit_transform(X)

In [ ]:
labelencoder_y=LabelEncoder()
y=labelencoder_y.fit_transform(y)

In [ ]:
y

In [ ]:
#Spliting dataset into training set and testing test

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


In [ ]:
# Fitting K-NN to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
